In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt


# What are we dealing with?

In [ ]:
train_df = pd.read_csv('../input/sorghum-id-fgvc-9/train_cultivar_mapping.csv', index_col = 'image')

In [ ]:
len(train_df)

So we have about 23k images to train on.

Turns out some images are missing:

In [ ]:
import os
images_present = os.listdir('../input/sorghum-id-fgvc-9/train_images')
len(images_present)

In [ ]:
# Only keep the files that exist
train_df = train_df.loc[images_present]

In [ ]:
train_df.head()

For each image we have a classification and nothing else.

In [ ]:
classes = train_df['cultivar'].unique()
classes

In [ ]:
train_df['cultivar'].value_counts()

For each class we have unequal number of training samples.

In [ ]:
print(train_df['cultivar'].value_counts().min())
print(train_df['cultivar'].value_counts().max())

The number of training images per class ranges from 134 to 298

In [ ]:
sample_sub = pd.read_csv('../input/sorghum-id-fgvc-9/sample_submission.csv')
sample_sub.head()

We are predicting on some 24k images

In [ ]:
len(sample_sub)

And they are all there:

In [ ]:
test_images_present = os.listdir('../input/sorghum-id-fgvc-9/test')
len(test_images_present)

# Separate train and test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_df.index.values, train_df['cultivar'],
                                                    stratify=train_df['cultivar'], 
                                                    test_size=0.1)
y_train = y_train.values
y_test  = y_test.values

# Pytorch

In [ ]:
from PIL import Image
import torch
import random
import os

In [ ]:
package_paths = [
    '../input/pytorch-image-models/pytorch-image-models-master' 
]
import sys; 

for pth in package_paths:
    sys.path.append(pth)
    
import timm

In [ ]:
CLIP = None
device = 'cuda' if torch.cuda.is_available() else 'cpu'
train_bs = 8
valid_bs = 8
num_workers = 2
model_arch = 'resnet34'
n_class = 100
x_size = 224
y_size = 224
num_epochs = 5

In [ ]:
device

In [ ]:
dir_name = '../input/sorghum-id-fgvc-9/train_images/'

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [ ]:
seed_everything(42)

In [ ]:
from torchvision import transforms

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torch import Tensor
from torchvision.transforms import ToTensor

In [ ]:
def encode_labels(y):
    res = np.zeros(n_class, dtype = int)
    res[np.argmax(classes == y)] = 1
        
    return res

In [ ]:
class TrainDataset(Dataset):
    def __init__(self):
        super().__init__()
    
    def __len__(self):
        return len(X_train)

    def __getitem__(self, idx):
        name = X_train[idx]
        y = encode_labels(y_train[idx])
        x = Image.open(dir_name + name)
        x = train_transform(x)
        return (x,Tensor(y[:n_class]))
    
class ValidDataset(Dataset):
    def __init__(self):
        super().__init__()
    
    def __len__(self):
        return len(X_test)

    def __getitem__(self, idx):
        name = X_test[idx]
        y = encode_labels(y_test[idx])
        x = Image.open(dir_name + name)
        x = test_transform(x)
        return (x,Tensor(y))

In [ ]:
train = TrainDataset()
valid = ValidDataset()

In [ ]:
xx, yy = train[0]
print(yy)
plt.imshow(xx.transpose(2,0))

In [ ]:
train_dl = DataLoader(train, batch_size = train_bs, shuffle = True, num_workers = num_workers)
valid_dl = DataLoader(valid, batch_size = valid_bs, shuffle = False, num_workers = num_workers)

In [ ]:
import torch.nn as nn

In [ ]:
class OurModel(nn.Module):
    def __init__(self, model_arch, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained)
        #for param in self.model.parameters():
        #    param.requires_grad = False
        n_features = self.model.fc.in_features
        self.model.fc = nn.Linear(n_features, n_class)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
from torch.optim import Adam
from torch.nn import BCEWithLogitsLoss
model = OurModel(model_arch, pretrained = True)
model.to('cuda')
adam = Adam(model.parameters(), lr = 2e-4)
loss = BCEWithLogitsLoss()

In [ ]:
def train_one_epoch():
    total_loss = 0
    num_correct = 0
    num_total = 0
    
    TARGETS = []
    PREDS = []
    
    model.train()    
    for i, (x, y) in enumerate(train_dl):
        x = x.to(device)
        y = y.to(device)
        adam.zero_grad()
        logits = model(x)
        error = loss(logits, y)
        error.backward()
        adam.step()
        with torch.no_grad():
            total_loss += error * len(y)
            preds = (logits > 0).to(torch.long)
            preds_correct = preds == y
            num_correct += torch.sum(preds_correct)
            num_total += preds_correct.shape[0] * preds_correct.shape[1]
            
            PREDS += [logits.sigmoid()]
            TARGETS += [y.detach().cpu()]
    
    PREDS = torch.cat(PREDS).cpu().numpy()
    TARGETS = torch.cat(TARGETS).cpu().numpy()
            
    print('Training epoch done')
    print('acc: ', num_correct/num_total)
    print(f'Train loss: {total_loss/len(train_dl)}')
    return total_loss / len(train_dl)

In [ ]:
def valid_after_one_epoch():
    total_loss = 0
    num_correct = 0
    num_total = 0
    
    TARGETS = []
    PREDS = []

    model.eval()
    with torch.no_grad():
        for i, (x, y) in enumerate(valid_dl):
            x = x.to(device)
            y = y.to(device)
            logits = model(x)
            preds = (logits > 0).to(torch.long)
            preds_correct = preds == y
            num_correct += torch.sum(preds_correct)
            num_total += preds_correct.shape[0] * preds_correct.shape[1]
            
            PREDS += [logits.sigmoid()]
            TARGETS += [y.detach().cpu()]
            
            error = loss(logits, y)
            total_loss += error * len(y)
    
    PREDS = torch.cat(PREDS).cpu().numpy()
    TARGETS = torch.cat(TARGETS).cpu().numpy()

    print('acc: ', num_correct/num_total)
    print(f'Valid loss: {total_loss/len(valid_dl)}')
    return total_loss / len(valid_dl)

In [ ]:
train_loss = np.zeros(num_epochs)
valid_loss = np.zeros(num_epochs)

for i in range(num_epochs):
    print(f'Epoch {i}')
    train_loss[i] = train_one_epoch()
    valid_loss[i] = valid_after_one_epoch()

    torch.save(model.state_dict(),f'epoch_{i}.pth')